In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [6]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [7]:
iris = datasets.load_iris()

In [8]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [10]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [11]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [12]:
# alpha - множитель перед регуляризацией

# поиск по сетке - метод перебора с множеством параметров, 
# измеряем качество классификации, сравинить, найтим где качество макксимально.
# Т.е. это метод перебора на множестве параметров 
# собвственно он и называется поиском по сетке.
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [27]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [15]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [28]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 7.3 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [29]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, max_iter=9, penalty='l1',
              random_state=0)

In [30]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [20]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00360763, 0.00361011, 0.00254786, 0.00326126, 0.0013587 ,
        0.00121405, 0.00166738, 0.00123181, 0.00303023, 0.00232055,
        0.00364709, 0.00259161, 0.00246296, 0.00280385, 0.00294771,
        0.01460857, 0.00368927, 0.00266628, 0.00320837, 0.00200639,
        0.002986  , 0.00256147, 0.00106409, 0.0017009 , 0.00345845,
        0.00373969, 0.00159955, 0.00279255, 0.00240395, 0.00264068,
        0.00408895, 0.00342546, 0.00298095, 0.00200348, 0.00341566,
        0.00284994, 0.00309696, 0.00350392, 0.00302496, 0.00117097,
        0.00309987, 0.00292032, 0.00318301, 0.00201128, 0.0030232 ,
        0.00186799, 0.00241957, 0.00170534, 0.002704  , 0.00256846,
        0.00143452, 0.003546  , 0.00249619, 0.0026015 , 0.00245471,
        0.00292287, 0.00265996, 0.00302973, 0.00384092, 0.00292878,
        0.00288098, 0.00208211, 0.00382657, 0.00353184, 0.00351765,
        0.00244343, 0.00305514, 0.00196664, 0.0034308 , 0.00312169,
        0.00303788, 0.00240791,

In [22]:
grid_cv

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

#### Randomized grid search

In [32]:
# Можно выбирать параметры случайным облразом, это позволяет сузить область описка по параметрам.
# Можно откинуть неоптимальные параметры, т.е. параметры на которых доситкается наихудшие параметры подели.
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [37]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 751 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   estimator=SGDClassifier(random_state=0), n_iter=20,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                                        'loss': ['hinge', 'log',
                                                 'squared_hinge',
                                                 'squared_loss'],
                                        'max_iter': array([5, 6, 7, 8, 9]),
                                        'penalty': ['l1', 'l2']},
                   random_state=0, scoring='accuracy')

In [38]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}


In [35]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

#print check(current_time + '\n' + "Control sum: " + str(control_sum(current_time)))

2021-10-04 18:40:46
Control sum: 943
